In [1]:
from typing import Callable, Dict, List, Set, Tuple
import sys  
sys.path.insert(0, '../UTIL')
from type_hierarchy import TypeTaxonomy

In [2]:
#Knowledge base features
FEATURES_13="DEPTH_t"
FEATURES_14="CHILDREN_t"
FEATURES_15="SIBLINGS_t"

In [3]:
typeobj=TypeTaxonomy("../data/dbpedia_types.tsv")

In [4]:
#typeobj._types

In [5]:
def extract_features_13to15(typeobj:TypeTaxonomy,dp_type:str)-> Dict[str, float]:
    dp_types=list(typeobj._types.keys())
    SIBLINGS=[item for item in dp_types if typeobj.depth(item)==typeobj.depth(dp_type) and item!=dp_type]
    return {FEATURES_13:round(typeobj.depth(dp_type)/typeobj.max_depth(),3),
           FEATURES_14:len(typeobj.children(dp_type)),
           FEATURES_15:len(SIBLINGS)}

In [6]:
extract_features_13to15(typeobj,"dbo:MilitaryPerson")

{'DEPTH_t': 0.429, 'CHILDREN_t': 0, 'SIBLINGS_t': 209}